# Building an AI-Powered Document Retrieval System with Docling and Granite

*Using IBM Granite Models*

## Recipe Overview

Welcome to this Granite recipe, in this recipe, you'll learn to harness the power of advanced tools to build AI-powered document retrieval systems. It will guide you through:

- **Document Processing:** Learn to handle documents from various sources, parse and transform them into usable formats, and store them in vector databases using Docling.
- **Retrieval-Augmented Generation (RAG):** Understand how to connect large language models (LLMs) like Granite with external knowledge bases to enhance query responses and generate valuable insights.
- **LangChain for Workflow Integration:** Discover how to use LangChain to streamline and orchestrate document processing and retrieval workflows, enabling seamless interaction between different components of the system.

This recipe leverages three cutting-edge technologies:

1. **[Docling](https://docling-project.github.io/docling/):** An open-source toolkit for parsing and converting documents.
2. **[Granite](https://www.ibm.com/granite/docs/models/granite/):** A state-of-the-art LLM available via an [API](https://www.ibm.com/topics/api) through Replicate, providing robust natural language capabilities.
3. **[LangChain](https://github.com/langchain-ai/langchain):** A powerful framework for building applications powered by language models, designed to simplify complex workflows and integrate external tools seamlessly.

By the end of this recipe, you will:
- Gain proficiency in document processing and chunking.
- Integrate vector databases to enhance retrieval capabilities.
- Utilize RAG to perform efficient and accurate data retrieval for real-world applications.

This recipe is designed for AI developers, researchers, and enthusiasts looking to enhance their knowledge of document management and advanced NLP techniques.



## Prerequisites

- Familiarity with Python programming.
- Basic understanding of large language models and natural language processing concepts.


## Step 1: Setting up the environment

Install dependencies.

In [1]:
%pip install "git+https://github.com/ibm-granite-community/utils.git" \
    transformers \
    langchain_community \
    'langchain_huggingface[full]' \
    langchain_milvus \
    docling \
    replicate

  Cloning https://github.com/ibm-granite-community/utils.git to /tmp/pip-req-build-hq41u2hz
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils.git /tmp/pip-req-build-hq41u2hz
  Resolved https://github.com/ibm-granite-community/utils.git to commit 19b0757d88cb6d052c53f4dbe04f3ea3f36cf011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.1/195.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 18.1 MB/s eta 0:00:00
   ━━━━

## Step 2: Selecting System Components

### Choose your Embeddings Model

Specify the model to use for generating embedding vectors from text. Here we will be using one of the new [Granite Embeddings models](https://huggingface.co/collections/ibm-granite/granite-embedding-models-6750b30c802c1926a35550bb)

To use a model from another provider, replace this code cell with one from [this Embeddings Model recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Embeddings_Models.ipynb).

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer

embeddings_model_path = "ibm-granite/granite-embedding-30m-english"
embeddings_model = HuggingFaceEmbeddings(
    model_name=embeddings_model_path,
)
embeddings_tokenizer = AutoTokenizer.from_pretrained(embeddings_model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Use the Granite model

Select a Granite model from the [`ibm-granite`](https://replicate.com/ibm-granite) org on Replicate. Here we use the Replicate Langchain client to connect to the model.

To get set up with Replicate, see [Getting Started with Replicate](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Getting_Started/Getting_Started_with_Replicate.ipynb).

To connect to a model on a provider other than Replicate, substitute this code cell with one from the [LLM component recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_LLMs.ipynb).

In [4]:
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

model_path = "ibm-granite/granite-3.3-8b-instruct"
model = Replicate(
    model=model_path,
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),
    model_kwargs={
        "max_tokens": 1000, # Set the maximum number of tokens to generate as output.
        "min_tokens": 100, # Set the minimum number of tokens to generate as output.
    },
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

REPLICATE_API_TOKEN not found in Google Colab secrets.
Please enter your REPLICATE_API_TOKEN: ··········


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

Now that we have the model downloaded, let's try asking it a question

In [5]:
from ibm_granite_community.langchain import TokenizerChatPromptTemplate

query = "Who won in the Pantoja vs Asakura fight at UFC 310?"

# Create a Granite prompt for question-answering
prompt_template = TokenizerChatPromptTemplate.from_template(template="{input}", tokenizer=tokenizer)

chain = prompt_template | model

output = chain.invoke({"input": query})

print(output)

In the Pantoja vs Asakura fight at UFC 310, Jussier Formiga "Pantera" defeated Mamoru Asakura via unanimous decision in the flyweight division. The event took place on October 8, 2011. Formiga dominated all three rounds, showcasing his grappling skills and control, ultimately securing the victory.

Here's a brief summary of the fight:

1. Round 1: Formiga controlled the pace, landing leg kicks and takedowns. Asakura struggled to find his rhythm and was unable to threaten Formiga's dominance.
2. Round 2: Formiga continued to dictate the fight with his grappling, taking Asakura down multiple times and controlling him on the ground. Asakura briefly threatened with a guillotine choke but couldn't secure it.
3. Round 3: Formiga maintained his lead, keeping Asakura at bay with strikes and takedowns. Asakura attempted a last-minute comeback with strikes, but Formiga defended well and maintained control.

The judges' scorecards reflected Formiga's clear dominance, with scores of 30-27 across t

Now, I know that UFC 310 happened in 2024, and this does not seem to be the right Pantoja. The model doesn't seem to know the answer but at least understands that this matchup did not occur. Let's see if it has some specific UFC rules info.

In [6]:
query1 = "How much weight allowance is allowed in non championship fights in the UFC?"

output = chain.invoke({"input": query1})

print(output)

In the UFC, non-championship fights typically adhere to a weight allowance of 2.5% of the lighter fighter's official weigh-in weight. This rule is in place to ensure fairness in matchups, allowing fighters in higher weight classes to compete against those in lower weight classes, provided they stay within this weight allowance. Here's a breakdown:

1. **Lighter fighter's official weigh-in weight x 2.5% = Weight allowance**

For example, if a featherweight (145 lbs) is fighting a lightweight (155 lbs), the lightweight can weigh up to 159.125 lbs (155 lbs x 1.025) at the time of weigh-ins to be considered within the 2.5% allowance.

It's essential to note that this rule applies to non-title fights, and champions usually have stricter weight limits for title defenses or challengers. Additionally, the UFC can make exceptions or adjustments to these rules for specific fights, as determined by the organization. Always refer to the official UFC Unified Rules of MMA for the most accurate and u

Based on the official UFC rules, this is also incorrect. Let's try getting some documents that contains this information for the model.

### Choose your Vector Database

Specify the database to use for storing and retrieving embedding vectors.

To connect to a vector database other than Milvus, replace this code cell with one from [this Vector Store recipe](https://github.com/ibm-granite-community/granite-kitchen/blob/main/recipes/Components/Langchain_Vector_Stores.ipynb).

In [7]:
from langchain_milvus import Milvus
import tempfile

db_file = tempfile.NamedTemporaryFile(prefix="milvus_", suffix=".db", delete=False).name
print(f"The vector database will be saved to {db_file}")

vector_db = Milvus(
    embedding_function=embeddings_model,
    connection_args={"uri": db_file},
    auto_id=True,
    enable_dynamic_field=True,
    index_params={"index_type": "AUTOINDEX"},
)

The vector database will be saved to /tmp/milvus_adc3iooh.db


## Step 3: Building the Vector Database

In this example, from a set of source documents, we use [Docling](https://docling-project.github.io/docling/) to convert the documents into text and then split the text into chunks, derive embedding vectors using the embedding model, and load it into the vector database. Creating this vector database will allow us to easily search across our documents, enabling us to use RAG.

### Use Docling to download the documents, convert to text, and split into chunks

Here we have found a website that gives us information on UFC 310, as well as a PDF of the official UFC rules. Below, we will see that Docling can both convert and chunk the two documents.

In [8]:
# Docling imports
from docling.document_converter import DocumentConverter
from docling_core.transforms.chunker.hybrid_chunker import HybridChunker
from docling_core.types.doc.labels import DocItemLabel
from langchain_core.documents import Document

# Here are our documents, feel free to add more documents in formats that Docling supports
sources = [
    "https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura",
    "https://media.ufc.tv/discover-ufc/Unified_Rules_MMA.pdf",
]

converter = DocumentConverter()

# Convert and chunk out documents
doc_id = 0
texts: list[Document] = [
    Document(page_content=chunk.text, metadata={"doc_id": (doc_id:=doc_id+1), "source": source})
    for source in sources
    for chunk in HybridChunker(tokenizer=embeddings_tokenizer).chunk(converter.convert(source=source).document)
    if any(filter(lambda c: c.label in [DocItemLabel.TEXT, DocItemLabel.PARAGRAPH], iter(chunk.meta.doc_items)))
]

print(f"{len(texts)} document chunks created")

Token indices sequence length is longer than the specified maximum sequence length for this model (524 > 512). Running this sequence through the model will result in indexing errors


23 document chunks created


In [9]:
# Print all created documents
for document in texts:
    print(f"Document ID: {document.metadata['doc_id']}")
    print(f"Source: {document.metadata['source']}")
    print(f"Content:\n{document.page_content}")
    print("=" * 80)  # Separator for clarity

Document ID: 1
Source: https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura
Content:
- Events
- Upcoming
- Past
- Tickets
- VIP Experiences
- Group Sales
- UFC Travel Deals
- Road to UFC
- Dana White's Contender Series
- Rankings
- Athletes
- All Athletes
- Hall of Fame
- Record Book
- News
- UFC
- Connect
- Newsletter
- UFC Fight Club
- UFC Apex
- Find a Gym
- EA Sports UFC 5
- Betting Odds
- Watch
- How to Watch
- Find a Bar
- UFC Fight Pass
- UFC Video Archive
- PODCASTS
- SHOP
- VENUM
- Apparel
- UFC COLLECTIBLES
- UFC STRIKE
- WHAT'S NEW
- Thorne Performance Solutions
PreviousNext
Results
Document ID: 2
Source: https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura
Content:
See The Fight Results, Watch Post-Fight Interviews With The Main Card Winners And More From UFC 310: Pantoja vs Asakura, Live From T-Mobile Arena In Las Vegas, Nevada
By E. Spencer Kyte, On X @spencerkyte
• Dec. 8, 2024
Th

### Populate the vector database

NOTE: Population of the vector database may take over a minute depending on your embedding model and service.

In [11]:
ids = vector_db.add_documents(texts)
print(f"{len(ids)} documents added to the vector database")

23 documents added to the vector database


## Step 4: RAG with Granite

Now that we have succesfully converted our documents and vectorized them, we can set up out RAG pipeline.

### Retrieve relevant chunks



Here we will test the as_retriever method to search through our newly created vector database for chunks that are relevant to our original query



In [12]:
retriever = vector_db.as_retriever()

docs = retriever.invoke(query)
print(docs)

[Document(metadata={'pk': 460044788642349073, 'doc_id': 18, 'source': 'https://www.ufc.com/news/main-card-results-highlights-winner-interviews-ufc-310-pantoja-vs-asakura'}, page_content='Cancel\nAlexandre Pantoja Post-Fight Interview | UFC 310\n/\nLive\nThe champion came out aggressively to start the fight, seeking to bring the action to Asakura. He stung the newcomer in the first and was happy to exchange through much of the first two rounds before jumping on Asakura’s back and sinking him to the canvas. From there, Pantoja attacked the choke and secured the finish.\nThat’s now three consecutive successful title defenses for the Brazilian, and arguably the most impressive effort to date inside the Octagon. The American Top Team representative remains a cut above the competition in the 125-pound weight class, and heads into next year riding a seven-fight winning streak.\nPrelim Results | Official Scorecards\nUFC 310: Pantoja vs Asakura took place live from T-Mobile Arena in Las Vegas, 

Looks like it pulled some chunks that would have the information we are looking for. Let's go ahead and contruct our RAG pipeline.

### Create the prompt for Granite

Next, we construct the prompt pipeline. This creates the prompt which holds the retrieved chunks from out previous search and feeds this to the model as context for answering our question.

In [13]:
from ibm_granite_community.langchain import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Assemble the retrieval-augmented generation chain
combine_docs_chain = create_stuff_documents_chain(
    llm=model,
    prompt=prompt_template,
)
rag_chain = create_retrieval_chain(
    retriever=vector_db.as_retriever(),
    combine_docs_chain=combine_docs_chain,
)

### Generate a retrieval-augmented response to a question

The pipeline uses the query to locate documents from the vector database and use them as context for the query.

In [14]:
output = rag_chain.invoke({"input": query})

print(output['answer'])

Alexandre Pantoja won the fight by submission in the second round.

Reference(s):
document_18
document_2
document_16
document_4
(Note: The answer is consistent across all documents, providing a strong confirmation of the answer.)

(The specific details of the submission are not mentioned in the documents, only that Pantoja won by submission.)
(There is no disagreement in the documents regarding the winner.)
(All documents provide information about the Pantoja vs Asakura fight and confirm Pantoja's victory.)


Awesome! It looks like the model figured out our first question. Let's see if it figure out the rule we were looking for.

In [15]:
output = rag_chain.invoke({"input": query1})

print(output['answer'])

In non-championship fights, there is a 1 pound weigh allowance in the UFC. This is according to the regulations outlined in document 21. The document specifies the weight classes for mixed martial arts contests or exhibitions, and then adds, "In non-championship fights, there shall be allowed a 1 pound weigh allowance." {"document_id": "1"} This means that fighters in non-championship bouts are permitted to weigh up to 1 pound more than the maximum weight for their respective class.


Awesome! We can now see that we have created a pipeline that can successfully leverage knowledge from multiple document types for generation.

## Next Steps

- Explore advanced RAG workflows for other industries
- Experiment with other document types and larger datasets.
- Optimize prompt engineering for better Granite responses.

Thank you for using this recipe!